# Implementing the Yann LeCun's Le-Net architecture
> "The implementation is not about getting the best accuracy"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

## Load the data

Load train and test data in memory

In [ ]:
file_path = '../input/digit-recognizer/'

train_data = pd.read_csv(file_path + 'train.csv')

In [ ]:
X = np.array(train_data.iloc[:,1:])
y = train_data.iloc[:,0]
labels = pd.get_dummies(y)
labels = np.array(labels)

In [ ]:
train_data = []
for i in X:
    train_data.append(i.reshape(28,28,1))
train_data = np.array(train_data)

# Splitting Data

In [ ]:
val_data = train_data[:2000]
val_labels = labels[:2000]

test_data = train_data[2000:2*2000]
test_labels = labels[2000:2*2000]

train_data = train_data[4000:]
train_labels = labels[4000:]

In [ ]:
train_labels.shape

In [ ]:
import random
random_indices = random.sample(range(0, len(train_data)), 9)

for i, idx in enumerate(random_indices):
    img = train_data[idx]
    label = y[4000+idx]
    
    plt.subplot(330 + 1 + i)    
    plt.tick_params(left=False,
            bottom=False,
            labelleft=False,
            labelbottom=False)
    
    plt.title('Label is {label}'.format(label=label))
    plt.tight_layout()
    plt.imshow(img, cmap='gray') 
    
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train_data[i])
    plt.tight_layout()
plt.show()

## Getting out Features ready

The LeNet architecture accepts a 32x32 pixel images as input, mnist data is 28x28 pixels. We simply pad the images with zeros to overcome that.

In [ ]:
# Pad images with 0s
train_data = np.pad(train_data, ((0,0),(2,2),(2,2),(0,0)), 'constant')
val_data = np.pad(val_data, ((0,0),(2,2),(2,2),(0,0)), 'constant')
test_data = np.pad(test_data, ((0,0),(2,2),(2,2),(0,0)), 'constant')

In [ ]:
train_data.shape
# The data is now ready

Original Le-Net architecture

![](https://drek4537l1klr.cloudfront.net/elgendy/v-3/Figures/05_01.png)

#### Input
    32x32x1 pixels image

#### Architecture
* **Convolutional #1** outputs 28x28x6
    * **Activation** any activation function, we will `relu`

* **Pooling #1** The output shape should be 14x14x6.

                        
* **Convolutional #2** outputs 10x10x16.
     * **Activation** any activation function, we will `relu`
    

* **Pooling #2** outputs 5x5x16.
    * **Flatten** Flatten the output shape of the final pooling layer

* **Fully Connected #1** outputs 120
    * **Activation** any activation function, we will `relu`

* **Fully Connected #2** outputs 84
    * **Activation** any activation function, we will `relu`

* **Fully Connected (Logits) #3** outpute 10


In [ ]:
model = keras.Sequential()

model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(32,32,1)))
model.add(layers.AveragePooling2D())

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(units=120, activation='relu'))

model.add(layers.Dense(units=84, activation='relu'))

model.add(layers.Dense(units=10, activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import Adam
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])

In [ ]:
# define the type of augmentation techniques we will apply.
train_datagen = ImageDataGenerator(
    rescale =1/255,
    shear_range=10,
    zoom_range = 0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=20,
    fill_mode = 'nearest',
)
val_datagen = ImageDataGenerator(
    rescale =1/255,
#     shear_range=10,
#     zoom_range = 0.2,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     rotation_range=20,
#     fill_mode = 'nearest',
)
test_datagen = ImageDataGenerator(
    rescale =1/255,
)


In [ ]:
train_generator=train_datagen.flow(
    train_data,
    train_labels,
    batch_size= 128,
)
validation_generator = val_datagen.flow(
        val_data,
        val_labels,
        batch_size=128,
)
test_generator = test_datagen.flow(
    test_data,
    test_labels,
    batch_size=128,
)

In [ ]:
history = model.fit(train_generator,
                    epochs=30,
                    steps_per_epoch = 38000 // 128,
                    validation_data = validation_generator,
                    validation_steps = 2000 // 128)

In [ ]:
model.evaluate(validation_generator)[1]

In [ ]:
model.evaluate(test_generator)[1]

In [ ]:
sub_data = pd.read_csv(file_path + 'test.csv')
test = np.reshape(np.array(sub_data), (sub_data.shape[0], 28, 28, 1))
test = np.pad(test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

In [ ]:
prediction = model.predict_classes(test).reshape(-1,1)
out = [{'ImageId': i+1, 'Label': prediction[i][0]} for i in range(len(prediction))]

In [ ]:
pd.DataFrame(out).to_csv('submission2.csv', index=False)